# EOEPCA Resource Registration Validation and Usage Notebook

## Registration API

In [ ]:
import os
import requests
import json
from datetime import datetime
from pathlib import Path
from urllib.parse import urlparse

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [ ]:
load_eoepca_state()

In [ ]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_registration_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-api.{platform_domain}'

# If you have a self-signed CA certificate for your cluster then this can be useful to avoid TLS errors below.
# First this certificate needs to be trusted, for example
#  kubectl get secret -n cert-manager eoepca-ca-secret -o yaml -o jsonpath='{.data.ca\.crt}' | base64 -d > /usr/local/share/ca-certificates/eoepca-local-ca/eoepca-local-ca.crt
#  ln -s /usr/local/share/ca-certificates/eoepca-local-ca/eoepca-local-ca.pem /etc/ssl/certs/
#  update-ca-certificates
os.environ["REQUESTS_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt"
verify_tls=True

print(f"Resource Registration API: {resource_registration_domain}")

### Validate Resource Registration API Endpoints

In [ ]:
endpoints = [
    ("Landing Page", resource_registration_domain),
    ("Swagger UI", f"{resource_registration_domain}/openapi?f=html"),
    ("Conformance Declaration", f"{resource_registration_domain}/conformance"),
    ("Processes List", f"{resource_registration_domain}/processes"),
    ("Jobs Endpoint", f"{resource_registration_domain}/jobs")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

### Inspect Available Processes

In [ ]:
processes_url = f"{resource_registration_domain}/processes"
processes_response = requests.get(processes_url)
processes = processes_response.json().get("processes", [])
print(f"Found {len(processes)} processes.")
for idx, process in enumerate(processes, 1):
    # If you want complete information about each process:
    # print(idx, json.dumps(process, indent=2))
    print(f"{idx}: {process['id']:<20} {process['version']:<15} {process['title']}")

### Registering Remote STAC Collection

In [ ]:
register_url = f"{resource_registration_domain}/processes/register/execution"

payload = {
    "inputs": {
        "type": "collection",
        "source": {"rel": "collection", "href": "https://raw.githubusercontent.com/james-hinton/temp-data-store/refs/heads/main/stac-collection.json"},
        "target": {"rel": "https://api.stacspec.org/v1.0.0/core", "href": f"https://resource-catalogue.{platform_domain}/stac"}
    }
}
response = requests.post(register_url, json=payload)
registration_result = response.json()
job_location = response.headers["Location"]
print(json.dumps(registration_result, indent=2))
print(job_location)

In [ ]:
# Confirm that the registration was successful
collection_href = registration_result["resource-and-data-catalogue-link"]['href']

response = requests.get(collection_href)
print(f"Collection successfully registered at {collection_href}")

### Verify Registration Job Status

In [ ]:
# A paged list of jobs can be obtaioned at f"{resource_registration_domain}/jobs", but
# here we go straight to the job URL returned above.
response = requests.get(job_location)
job = response.json()

print(f"Job fetch status code: {response.status_code}")
print(json.dumps(job, indent=2))

### Validate Registered Dataset in Resource Discovery

In [ ]:
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'
metadata_collections_url = f"{resource_discovery_domain}/collections"

response = requests.get(metadata_collections_url)
collections = response.json().get("collections", [])

collection_ids = [collection.get("id") for collection in collections]
expected_collection_id = "S2MSI2A"

if expected_collection_id in collection_ids:
    print(f"✅ Dataset '{expected_collection_id}' successfully registered and discoverable!")
    print(f"View it here: {metadata_collections_url}/{expected_collection_id}")
else:
    print(f"❌ Dataset '{expected_collection_id}' registration failed or collection not discoverable.")

### Deregistering a STAC Collection

In [ ]:
deregister_url = f"{resource_registration_domain}/processes/deregister/execution"
payload = {
    "inputs": {
        "id": "S2MSI2A",
        "target": {
            "rel": "https://api.stacspec.org/v1.0.0/core",
            "href": f"https://resource-catalogue.{platform_domain}/stac"
        },
        "rel": "collection"
    }
}
response = requests.post(deregister_url, json=payload)
deregistration_job_location = response.headers["Location"]

print(response.json())

### Verify Deregistration Job Status

In [ ]:
response = requests.get(deregistration_job_location)
job = response.json()

print(f"Job fetch status code: {response.status_code}")
print(json.dumps(job, indent=2))

### Verify Collection Deregistration

In [ ]:
collection_url = f"{metadata_collections_url}/{expected_collection_id}"
response = requests.get(collection_url)
if response.status_code == 404:
    print(f"✅ Dataset '{expected_collection_id}' was deregistered")
elif response.status_code == 200:
    print(f"❌ Dataset '{expected_collection_id}' still registered")
else:
    print(f"❌ Unexpected response from {collection_url}: {response.status_code} {response.text}")

## Registration Harvester

In [ ]:
flowable_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-harvester-api.{platform_domain}'
flowable_docs_url = f"{flowable_domain}/flowable-rest/docs/"
response = requests.get(flowable_docs_url)

print(f"Flowable Swagger UI status code: {response.status_code}")
response.url

In [ ]:
from requests import Session
from requests.auth import HTTPBasicAuth
import json
import requests
import os

flowable_admin_user = os.environ.get("FLOWABLE_ADMIN_USER", "eoepca")
flowable_admin_password = os.environ.get("FLOWABLE_ADMIN_PASSWORD", "eoepca")
flowable_session = Session()
flowable_session.auth = HTTPBasicAuth(flowable_admin_user, flowable_admin_password)

### List Flowable Deployments

In [ ]:
deployments_url = f"{flowable_domain}/flowable-rest/service/repository/deployments"
response = flowable_session.get(deployments_url)
deployments = response.json().get("data", [])

print(f"Found {len(deployments)} deployments.")

if deployments:
    latest_deployment = deployments[0]
    deployment_id = latest_deployment.get("id")
    for idx, deployment in enumerate(deployments, 1):
        print("%-2s %-25s deployed at: %-30s with id: %s" % (idx, deployment['name'], deployment['deploymentTime'], deployment["id"]))

### Deploy Landsat Workflows

In [ ]:
landsat_bpmn_files = [
    "https://raw.githubusercontent.com/EOEPCA/registration-harvester/refs/heads/main/workflows/landsat.bpmn", 
    "https://raw.githubusercontent.com/EOEPCA/registration-harvester/refs/heads/main/workflows/landsat-scene-ingestion.bpmn"
]

for bpmn in landsat_bpmn_files:
    bpmn_file = {os.path.basename(bpmn): requests.get(bpmn).text}
    response = flowable_session.post(deployments_url, files=bpmn_file)
    print(f"Sucessfully deployed workflow: {os.path.basename(bpmn)} with status code: {response.status_code}")

### List Flowable Deployed Processes

In [ ]:
process_definitions_url = f"{flowable_domain}/flowable-rest/service/repository/process-definitions"
print(f"Flowable process definitions URL: {process_definitions_url}")

response = flowable_session.get(process_definitions_url)
processes = response.json()["data"]

for idx, process in enumerate(processes, 1):
    print("%-2s %-28s version: %-5s id: %s" % (idx, process['name'], process['version'], process['id']))
    if process["name"] == "Landsat Workflow":
        landsat_process_id = process["id"]

### Execute `Landsat Workflow` Process

In [ ]:
query = json.dumps({ "created": { "gte": "2024-12-13T10:00:00.000000Z", "lt": "2024-12-13T11:00:00.000000Z" } })
variables = [
    {"name": "datetime_interval", "type": "string", "value": "2024-11-13T10:00:00Z/2024-11-13T11:00:00Z"},    
    {"name": "collections", "type": "string", "value": "landsat-c2l2-sr"},
    {"name": "bbox", "type": "string", "value": "-7,46,3,52"},
    # {"name": "query", "type": "string", "value": query},
]

# Create the JSON body for HTTP request which triggers the workflow
body = {}
body["processDefinitionId"] = landsat_process_id
body["variables"] = variables

process_instances_url = f"{flowable_domain}/flowable-rest/service/runtime/process-instances"
response = flowable_session.post(process_instances_url, json=body)
print(f"POST {process_instances_url}")
print(json.dumps(response.json(), indent=2))

### Check Process Instances

In [ ]:
process_instances_url = f"{flowable_domain}/flowable-rest/service/runtime/process-instances"
print(f"GET {process_instances_url}")
response = flowable_session.get(process_instances_url)

process_instances = response.json().get("data", [])
print(f"Found {len(process_instances)} workflow instances.")
for idx, process in enumerate(process_instances, 1):
    print(f"{idx} {process['processDefinitionName']:<25} started at: {process['startTime']:<25} id: {process['id']}")

### View Harvester Logs

In [ ]:
# If a working Kubernetes client is not available to the Notebook then you should
# run this from a shell with access to your cluster.
! kubectl logs -n resource-registration deployment.apps/landsat-harvester-worker | tail -n 200

### Delete Workflow Instances

In [ ]:
process_instances_url = f"{flowable_domain}/flowable-rest/service/runtime/process-instances"
print(f"Fetch workflow instances: GET {process_instances_url}")
response = flowable_session.get(process_instances_url)
process_instances = response.json().get("data", [])
print(f"Found {len(process_instances)} workflow instances.")
for process in process_instances:
    # In development environments using non-standard ports, Flowable will return URLs without
    # the correct port. This corrects the URL.
    process_url = urlparse(process["url"])
    if ":" in flowable_domain:
        process_url = process_url._replace(netloc=urlparse(flowable_domain).netloc).geturl()
    
    print(f'DELETE: {process["id"]}: {process_url}')
    flowable_session.delete(process_url)

### Delete Deployments

In [ ]:
print(f"Fetch deployments: GET {deployments_url}")
response = flowable_session.get(deployments_url)
deployments = response.json().get("data", [])
print(f"Found {len(deployments)} deployments.")
for d in deployments:
    if not d["name"] == 'Demo processes':
        # Flowable generates links without a port number even when it's listening on a non-standard port.
        # This corrects for that.
        deployment_url = d["url"]
        if ":" in flowable_domain:
            deployment_url = urlparse(deployment_url)._replace(netloc=urlparse(flowable_domain).netloc).geturl()

        print(f'DELETE: {d["id"]}: {deployment_url}')
        flowable_session.delete(deployment_url)
    else:
        print(f'Retaining: {d["id"]}: {d["url"]}')        